# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height    

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 航太維修商機正夯　外資增資長異逾17億元搶市

▲經濟部投審會通過僑外投資案，包括長異發動機維修公司、LAMIGO桃猿隊等投資案。（圖／記者湯興漢攝）
記者林淑慧／台北報導
航空微利時代，修飛機比開飛機更能賺。經濟部表示，近期通過模里西斯商GE MAURITIUS INFRASTRUCTURE HOLDINGS LTD.以新臺幣逾17億元，增資長異發動機維修公司，將加碼在台投資發動機維修業務，搶攻航太高毛利商機。
經濟部投審會日前召開1183次委員會議，會中計核准及備查重大投資案件計11件，其中僑外投資案達6件、對外投資2件，對大陸投資案為3件。
蔡政府力推綠能政策，帶動風電相關投資。英屬開曼群島商STONEPEAK OCEANVIEW （CAYMAN） HOLDINGS COMPANY分別匯入新臺幣50億7,272萬8,500元、21億2,813萬2,000元，主要用以直、間接增資海能國際投資股份有限公司（實行後股權比率分別為75％、25％），作為海能風場相關設施建造費用。
經濟部官員指出，丹麥商ORSTED WIND POWER TW HOLDING A/S以新臺幣50億元增資沃旭能源股份有限公司（實行後股權比例100％），作為大彰化風電建設有關設備及服務之用。
另外，新加坡商RAKUTEN ASIA PTE. LTD.也以新臺幣12億元增資台灣樂天市場股份有限公司，主要用以轉投資大高熊育樂股份有限公司，經營中華職棒大聯盟LAMIGO桃猿隊，帶動台灣運動休閒育樂業發展可期。
----------------------------------------------------------------------

[1] 新台幣續強！早盤升值逾1角　升破30.1元兌1美元

▲ 新台幣兌美元今天開盤升值0.3分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（30）天開盤升值0.3分、以30.2兌1美元開出。開盤持續升值，並升值逾1角，最高來到30.095元兌1美元。
新台幣兌美元上週五（27日）收盤收在30.203兌1美元，終結連續三個交易日升值；今天開盤以30.2元開出、升值0.3分。

▲明年報綜所稅有3利多，相關民眾可以檢附證明運用，達到減稅的小確幸。（圖／記者湯興漢攝）
記者吳靜君／台北報導
政府提高4大扣除額，包含標準扣除額、薪資扣除額、身心障礙扣除額以及幼兒學前扣除額，民眾今年報稅時很有感之外。迎接新的一年到來，政府再增加3大減稅利多，民眾可以準備相關的檢附文件、明（2020）年報稅時檢視是否可以運用，讓自己的小確幸溫暖再升級。
一，林若亞條款上路　薪資扣除額可以核實減稅
今年7月已經公布修正所得稅法，未來薪資扣除額可以二擇一，一為定額減除20萬元；或者是可以選擇核實扣除（又稱為林若亞條款），其適用範圍及認定辦法如下包含職業專用的服裝費，包含購置、租用及清潔費等；職業所需的進修費用；以及職業所需的工具，其效能非2年內所能耗竭且支出超過8萬元者，前述三項可以扣除的各為所得的3%為限。
北區國稅局舉例，以某甲的薪資收入900萬元，今年若自費購買設計軟體90萬元，甲君可列報職業上工具支出上限為27萬元（900萬元×3%）低於實際支出按平均法分3年免列殘值可攤提折舊30萬元，但高於當年度薪資所得特別扣除額20萬元，甲君可檢附相關證明文件自其薪資收入900萬元中核實減除必要費用27萬元，多了7萬元的可扣除額，申報薪資所得就變成了873萬元。
二，長照扣除額　每人每年12萬元
納稅義務人、配偶或者是受扶養的親屬，若有符合衛福部所公告須長期照顧的身心失能者，明年申報所得稅時，每人多12萬元可列報長照扣除額。
但是民眾應該檢附的證明文件包含，如果有聘僱外籍看護工，就可以檢附聘僱許可函；如果在家自行照顧者，可以檢附病症失能診斷證明暨巴氏量表影本或者是若符合特定身心障礙重度等級項目或鑑定向度之一，就可附身心障礙證明、手冊等。
▲申請長照扣除額應檢附的文件。（圖／財政部提供）
如果符合長服法失能等級為第2級至第8級且使用長期照顧給付及支付基準服務者，只要檢附今年度使用服務的繳費收據影本、註記特約服務單位名稱、失能者姓名、身分證字號及失能等級等資料。
入住住宿式服務機構全年達90日者，檢附入住累計達90日的繳費收據影本，並註記機構名稱、住民姓名、身分證字號、入住期間及床位類型等資料。
國稅局洽相關單位　已歸戶者可以不用再行檢附
為了提升便民服務，國稅局已洽衛生福利部、勞動部及國軍退除役官兵輔導委員會等相關部會蒐集長照扣除額資料，納稅義務人在明（2020）年5月報綜